In [1]:
import json
import cv2
import numpy as np
from mediapipe import solutions

In [2]:
# Initialize Blazepose
mp_pose = solutions.pose
mp_drawing = solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, model_complexity=2)


In [1]:
"""
    Defining keypoints as nodes
"""

def init():
    angles_dict = {
        "elbow_left": 0,
        "elbow_right": 1,
        "knee_left": 2,
        "knee_right": 3,
        "hip_left": 4,
        "hip_right": 5,
    }
    return angles_dict

In [2]:
# Function to calculate angles
def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # First point
    b = np.array(b)  # Middle point
    c = np.array(c)  # Last point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360.0 - angle
    return angle

In [5]:
# Error margin check
def error_margin(control, value):
    """Check if the value is within the error margin of the control."""
    # if int(value) in range(control - 20, control + 21):
    #     return True

    if (value > control - 30 and value < control + 31):
        return True
    return False

In [6]:
# Check joint position and generate feedback
def check_joint(angles, joint_name, threshold, body_position):
    angles_dict = init()
    joint_index = angles_dict[joint_name]
    print(angles)
    print(joint_name, joint_index)

    if error_margin(threshold, angles[joint_index]):
        return None

    if angles[joint_index] > threshold:
        return f"Bring {' '.join(joint_name.split('_')[::-1])} closer to {body_position}."
    elif angles[joint_index] < threshold:
        return f"Put {' '.join(joint_name.split('_')[::-1])} further away from {body_position}."

    return None

In [7]:
# Generate feedback for a pose
def check_pose_angle(asana_name, angles, json_data):
    result = []

    # Get reference data for the specific asana
    pose_data = json_data[asana_name]

    joints = [
        ("elbow_left", "arm"),
        ("elbow_right", "arm"),
        ("knee_left", "calf"),
        ("knee_right", "calf"),
        ("hip_left", "pelvis"),
        ("hip_right", "pelvis"),
    ]

    for joint, body_part in joints:
        result.append(
            check_joint(
                angles,
                joint,
                pose_data[joint],
                body_part
            )
        )

    return [message for message in result if message is not None]

In [8]:
# Load reference pose data from JSON file
reference_data_path = "reference_pose_data.json"  # Replace with your JSON file path
with open(reference_data_path, 'r') as file:
    reference_data = json.load(file)

In [9]:
# Real-time pose correction
cap = cv2.VideoCapture(0)  

asana_selected = "goddess"  # Select the desired asana

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Flip the frame for a mirror effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # Draw landmarks
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
        )

        # Extract keypoints
        landmarks = results.pose_landmarks.landmark
        keypoints = {
            "left_shoulder": [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            "right_shoulder": [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                               landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            "left_elbow": [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
            "right_elbow": [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            "left_wrist": [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
            "right_wrist": [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
            "left_hip": [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
            "right_hip": [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            "left_knee": [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
            "right_knee": [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
            "left_ankle": [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
            "right_ankle": [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
        }
        
        # Calculate angles
        angles = [
            calculate_angle(keypoints["left_shoulder"], keypoints["left_elbow"], keypoints["left_wrist"]),
            calculate_angle(keypoints["right_shoulder"], keypoints["right_elbow"], keypoints["right_wrist"]),
            calculate_angle(keypoints["left_hip"], keypoints["left_knee"], keypoints["left_ankle"]),
            calculate_angle(keypoints["right_hip"], keypoints["right_knee"], keypoints["right_ankle"]),
            calculate_angle(keypoints["left_shoulder"], keypoints["left_hip"], keypoints["left_knee"]),
            calculate_angle(keypoints["right_shoulder"], keypoints["right_hip"], keypoints["right_knee"]),
        ]
        
        # Generate feedback
        feedback = check_pose_angle(asana_selected, angles, reference_data)
        
        # Display feedback on the frame
        y_position = 50
        for message in feedback:
            cv2.putText(frame, message, (10, y_position), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
            y_position += 30


    else:
        cv2.putText(frame, "No pose detected. Adjust your position.", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)

    # Display the frame
    cv2.imshow("Yoga Pose Correction", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
elbow_left 0
[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
elbow_right 1
[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
knee_left 2
[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
knee_right 3
[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
hip_left 4
[163.32705059226342, 139.50905665538374, 175.56168241238612, 175.30713443461403, 171.38746155542074, 176.16118196732683]
hip_right 5
[166.58689904737108, 105.68516143580747, 175.02346564848366, 2.2353548022373135, 171.99820778524074, 6.0759380220762145]
elbow_left 0
[166.58689904737108, 105.68516143580747, 175.02346564848366, 2.23